In [1]:
from openai import OpenAI
import openai

client = OpenAI(
    api_key='YOUR_KEY'
        )

In [ ]:
TEST_FILE_NAME = 'YOUR-TEST-FILE-PATH'

In [3]:
def query_gpt(messages):
    response = client.chat.completions.create(
    model="gpt-4",
    messages=messages,
    temperature=0,
    # top_p=0.95,
    # frequency_penalty=0,
    # presence_penalty=0,
    stop=None)
    answer=response.choices[0].message.content
    return answer

In [ ]:
system="Decide the status value for each medical problem event. Choose from present, absent, possible, conditional, hypothetical, not_patient. present: patient experienced or is experiencing; absent: patient has not or is not experiencing; possible: patient may be experiencing (denoted by terms like “probably” or “likely”); conditional: patient only experiences under specific conditions; hypothetical: patient may experience in the future; not_patient: not associated with the patient"
user='应该是双侧发病，考虑湿疹 medical problem event: 湿疹'
result=query_gpt([
    {"role":"system","content":system},
    {"role":"user","content":user}
])
result

'status value: present'

In [65]:
import json

with open( TEST_FILE_NAME, 'r', encoding='utf-8') as f:
    data = json.load(f)


In [62]:
## GET ASSERTION STATUS USING A GPT4 QUERY
import time
from tqdm import tqdm

max_attempts = 100  # Maximum number of attempts for each operation

for item in tqdm(data):
    for response in item['responses']:
        user_content = response["content_zh"]
        for output_item in response["UMLS_zh"]:
            if output_item["type"] == "Disease":
                query_content = user_content + " medical problem event: " + str(output_item["term"])
                attempts = 0  # Current attempt count
                while attempts < max_attempts:
                    try:
                        status_value = query_gpt([
                            {"role": "system", "content": system},
                            {"role": "user", "content": query_content}
                        ])
                        output_item["status"] = status_value
                        break  # Break out of the loop if successful
                    except TimeoutError as te:
                        attempts += 1
                        time.sleep(1)  # Wait a bit before retrying
                    except Exception as e:
                        attempts += 1
                        time.sleep(1)  # Wait a bit before retrying
                if attempts == max_attempts:
                    # Handle the case where all attempts fail
                    output_item["status"] = ''  # Or any other default/error value

100%|██████████| 100/100 [00:25<00:00,  3.98it/s]


In [63]:
status_counts = {status: 0 for status in ['present', 'absent', 'possible', 'conditional', 'hypothetical', 'not_patient']}
irregular_count = 0

for item in data:
    for response in item['responses']:
        for output_item in response["UMLS_zh"]:
            if output_item["type"] == "Disease":
                # Use .get() to avoid KeyError, and provide a default value '' (empty string)
                status_value = output_item.get("status", '').lower().split()
                status_value_filtered = ' '.join([word for word in status_value 
                                                if word in ['present', 'absent', 'possible', 'conditional', 'hypothetical', 'not_patient']])
                output_item["status"] = status_value_filtered
                
                if status_value_filtered in status_counts:
                    status_counts[status_value_filtered] += 1
                else:
                    irregular_count += 1

# Print the counts for each status
for status, count in status_counts.items():
    print(f"The total count for {status}: {count}")

print("The total number of irregular outputs: ", irregular_count)



The total count for present: 5
The total count for absent: 0
The total count for possible: 48
The total count for conditional: 0
The total count for hypothetical: 0
The total count for not_patient: 0
The total number of irregular outputs:  0


In [54]:
## rerun gpt query for irregular outputs, then rerun the cell above until The total number of irregular outputs is 0.
for item in tqdm(data):
    for response in item['responses']:
        user_content = response["content_zh"]
        for output_item in response["UMLS_zh"]:
            if (output_item["type"] == "Disease") and (output_item["status"] not in ['present', 'absent', 'possible', 'conditional', 'hypothetical', 'not_patient']):
                query_content = "Your output must be a single value in ['present', 'absent', 'possible', 'conditional', 'hypothetical', 'not_patient']: " + user_content + " medical problem event: " + str(output_item["term"])
                attempts = 0  # Current attempt count
                while attempts < max_attempts:
                    try:
                        status_value = query_gpt([
                            {"role": "system", "content": system},
                            {"role": "user", "content": query_content}
                        ])
                        output_item["status"] = status_value
                        break  # Break out of the loop if successful
                    except TimeoutError as te:
                        attempts += 1
                        time.sleep(1)  # Wait a bit before retrying
                    except Exception as e:
                        attempts += 1
                        time.sleep(1)  # Wait a bit before retrying
                if attempts == max_attempts:
                    # Handle the case where all attempts fail
                    output_item["status"] = ''  # Or any other default/error value

100%|██████████| 100/100 [00:01<00:00, 77.58it/s]


In [80]:
with open(TEST_FILE_NAME, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)